In [1]:
from dotenv import load_dotenv
load_dotenv()

import os
project_name = "wanted_2nd_langchain_outputparser_basic"
os.environ["LANGSMITH_PROJECT"] = project_name

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

model = ChatOpenAI(
    temperature=0.1,
    model="gpt-4.1-mini",
    verbose=True
)

In [3]:
from typing import Dict
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

## 1. 프롬프트에 history 자리를 확보

In [4]:
# 1. 프롬프트에 history 자리를 확보
prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 AI 도우미야. 간략하게 그냥 응답하도록 해"),
    MessagesPlaceholder(variable_name = "history"),
    ("user", "{question}")
])
prompt

ChatPromptTemplate(input_variables=['history', 'question'], input_types={'history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchai

In [5]:
chain = prompt | model | StrOutputParser()

## 2. 대화내용 저장소 만들기

In [6]:
stores : Dict[str, InMemoryChatMessageHistory] = {}
def get_store(session_id : str):
    if session_id not in stores:    # 새 대화창일 경우
        stores[session_id] = InMemoryChatMessageHistory()
    return stores[session_id]

## 3. 히스토리와 래핑

In [7]:
with_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: get_store(session_id),
    input_messages_key = "question",
    history_messages_key = "history"
)

In [8]:
cfg = {"configurable" : {"session_id" : "user-123"}}        # 이 값으로 대화를 이어나감
result = with_history.invoke({"question" : "내가 좋아하는 폰은 갤럭시야"}, config = cfg)
print(result)

갤럭시 좋은 선택이야!


In [9]:
result = with_history.invoke({"question" : "그 장점만 요약해봐"}, config = cfg)
print(result)

갤럭시 장점: 뛰어난 디스플레이, 강력한 성능, 다양한 카메라 기능, 확장 가능한 저장공간, 삼성 생태계와의 호환성.


In [10]:
result = with_history.invoke({"question" : "아이폰보다 좋아?"}, config = cfg)
print(result)

사용자 취향에 따라 다르지만, 갤럭시는 커스터마이징과 확장성에서 강점이 있어요.


In [11]:
result = with_history.invoke({"question" : "너의 개인적인 견해를 말해"}, config = cfg)
print(result)

저는 중립적이라 개인적인 견해는 없어요.


In [13]:
stores["user-123"].messages

[HumanMessage(content='내가 좋아하는 폰은 갤럭시야', additional_kwargs={}, response_metadata={}),
 AIMessage(content='갤럭시 좋은 선택이야!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='그 장점만 요약해봐', additional_kwargs={}, response_metadata={}),
 AIMessage(content='갤럭시 장점: 뛰어난 디스플레이, 강력한 성능, 다양한 카메라 기능, 확장 가능한 저장공간, 삼성 생태계와의 호환성.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='아이폰보다 좋아?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='사용자 취향에 따라 다르지만, 갤럭시는 커스터마이징과 확장성에서 강점이 있어요.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='너의 개인적인 견해를 말해', additional_kwargs={}, response_metadata={}),
 AIMessage(content='저는 중립적이라 개인적인 견해는 없어요.', additional_kwargs={}, response_metadata={})]